In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import datetime
import torch
from torch.nn import functional as F
import scipy.stats
import sklearn.datasets

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.save_model as sm
import curvvae_lib.architecture.load_model as lm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-o2o49o1c because the default path (/home/tsr42/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Dataset

In [2]:
foodname = "banana"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [3]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)
# actually this is all points, not just start points
start_points = all_points[:,:,1:].reshape(-1,7)
print("start_points_shape", start_points.shape)
# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
mean = np.mean(start_points, axis=0)
variance = np.var(start_points, axis=0)
position_std = np.sqrt(np.max(variance))
print("std of: ", position_std)
position_scaling = 1/position_std
rotation_scaling = 0.16 * position_scaling

start_points = start_points - mean
poss = position_scaling
rts = rotation_scaling
start_points = (start_points * np.array((poss,poss,poss,rts,rts,rts,rts)))
#start_points = start_points[:,3:]
print("shape before reducing dimension", all_points.shape)
print("dataset variance of:", np.var(start_points,axis=0))

We found 155 pickup attempts
start_points_shape (9920, 7)
std of:  0.17859106200728153
shape before reducing dimension (155, 64, 8)
dataset variance of: [0.00082463 0.00106736 0.00316102 0.00634536 0.0256     0.00564557
 0.00894256]


In [4]:
dataset = start_points
dataset

array([[ 5.30630546e-02, -7.39897730e-02,  6.99848376e-02, ...,
         5.77279855e-02,  3.91912119e-02,  2.56211944e-02],
       [ 4.82265267e-02, -6.74389532e-02,  6.44204345e-02, ...,
         5.95007100e-02,  3.93298113e-02,  2.38431718e-02],
       [ 4.42146147e-02, -6.03095820e-02,  5.96689720e-02, ...,
         6.08009003e-02,  3.93789762e-02,  2.20194015e-02],
       ...,
       [-9.03704476e-03,  1.13378370e-01,  8.72505381e-02, ...,
        -7.04652473e-02, -2.69143945e-02, -1.99310649e-01],
       [-2.72879079e-03,  1.19000039e-01,  1.06496165e-01, ...,
        -9.01796312e-02, -2.89765200e-02, -2.14355879e-01],
       [-1.67703832e-05,  1.42264068e-01,  1.38627073e-01, ...,
        -1.05488737e-01, -2.73580771e-02, -2.35037893e-01]])

In [5]:
np.random.seed(1)
train_indices = np.random.choice(len(dataset),100,replace=False)
test_indices = [i for i in range(len(dataset)) if i not in train_indices]
train_dataset=dataset[train_indices]
test_dataset=dataset[test_indices]
print(train_dataset.shape,test_dataset.shape)

(100, 7) (9820, 7)


In [6]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(train_dataset[:,0],train_dataset[:,1],train_dataset[:,2]);
ax.scatter(test_dataset[:,0],test_dataset[:,1],test_dataset[:,2]);

<IPython.core.display.Javascript object>

In [7]:
#fig = plt.figure()
#ax = plt.axes(projection='3d')
#ax.scatter(train_dataset[:,3],train_dataset[:,4],train_dataset[:,5]);
#ax.scatter(test_dataset[:,3],test_dataset[:,4],test_dataset[:,5]);

## 2D VAE Fit to Dataset

In [8]:
def LoadDataBatch(all_points, batchsize, passthroughdim, predictive, device):
    """Sample Pair of Points from Trajectory"""
    # all_points should be of dimension: num_trajectories, numtimesteps, 1+spatialdims
    traj_ids = np.random.choice(all_points.shape[0], batchsize)
    t1_ids = np.random.choice(all_points.shape[1], batchsize)
    if predictive:
        t2_ids = np.random.choice(all_points.shape[1], batchsize)
    else:
        t2_ids = t1_ids
    return (torch.tensor(all_points[traj_ids, t1_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t1_ids,:passthroughdim], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,:passthroughdim], dtype=torch.float).to(device))

In [9]:
class Loader(object):
    def __init__(self, all_points, batchsize, passthroughdim, epochnumbatches, predictive, device):
        self.all_points = all_points
        self.batchsize = batchsize
        self.passthroughdim = passthroughdim
        self.epochnumbatches = epochnumbatches
        self.predictive = predictive
        self.device = device
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n >= self.epochnumbatches:
            # https://docs.python.org/3/library/exceptions.html#StopIteration
            raise StopIteration
        self.n += 1
        return LoadDataBatch(self.all_points, self.batchsize, self.passthroughdim, self.predictive, self.device)
    

device = "cuda"
  

In [10]:
dataset_dim = dataset.shape[-1]
def make_vae(latent_dim):
    input_dim = dataset_dim 
    passthrough_dim = 0
    emb_layer_widths = [1000]
    recon_layer_widths = [1000]
    dtype = torch.float
    model = ptvae.FCPassthroughVAE(input_dim, passthrough_dim, latent_dim,
        emb_layer_widths, recon_layer_widths, dtype)
    return model

In [11]:
def error(vae, dataset):
    embeds = vae.encode(torch.tensor(dataset,dtype=torch.float).to(device),
                       torch.tensor(np.zeros((len(dataset),0)),dtype=torch.float).to(device)
                       )[0].detach().cpu().numpy()
    recons = vae.decode(torch.tensor(embeds,dtype=torch.float).to(device),
              torch.tensor(np.zeros((len(embeds),0)),dtype=torch.float).to(device))[0].detach().cpu()
    rmse = np.sqrt(np.mean(np.sum(np.square(dataset - recons.numpy()),axis=1)))
    return rmse

In [12]:
# train a CurvVAE model with given beta and gamma and return
# the string name of the saved model
def train_vae_model(beta,gamma,lr,epochs,latentdim,testname,train_dataset,test_dataset,device="cpu"):
    vae = make_vae(latent_dim=latentdim).to(device)
    num_epochs = epochs
    epsilon_scale_start = 0.5
    epsilon_scale = epsilon_scale_start
    savedir  = f'{testname}lat{latentdim}_lr{lr}_gamma{gamma}_es{epsilon_scale_start}_beta{beta}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    print(savedir)
    train_loader = Loader(train_dataset[:,np.newaxis,:], 256, 0,10,predictive=False, device=device)
    trainer = ppttrainer.PPTVAETrainer(vae, train_loader, beta=beta, device=device,
        log_dir=savedir, lr=lr, annealingBatches=0, record_loss_every=200, loss_func = "gaussian")
    epoch = 0
    traintesterrors = []
    for _ in range(num_epochs*3):
        epoch+=1
        # up num sample points to reduce variance of curvature measure
        embst = trainer.train(second_deriv_regularizer=secreg, curvature_regularizer=gamma, epsilon_scale = epsilon_scale, num_new_samp_points=512)
        is_error = embst[0]
        if is_error:
            raise Exception
        if epoch % 10 == 0:
            traintesterrors.append((epoch,error(vae,train_dataset),error(vae,test_dataset)))
    sm.save_fcpassthrough_vae(vae, savedir)
    print(savedir)
    return savedir,traintesterrors

In [13]:
test_dataset.shape

(9820, 7)

In [ ]:
testname = "overfitting/allposes"
all_models=[]
all_traintesterrors = []
for run in range(10):
    for lr in [0.00005,0.0001,0.00001]:
        for beta,gamma in [
                           [0.00001, 0.001  ],
                           [0.00001, 0.00001],
                           [0.001, 0],
                           [0.00001, 0],
                           [0.00001, 0.00005],
                           [0.00001, 0.0001 ],
                           [0.00001, 0.0002 ],
                           [0.00001, 0.0003 ],
                           [0.00001, 0.0005 ],
                           [0.00001, 0.0007 ],
                           [0.00001, 0.0008 ],
                           [0.00001, 0.0009 ],
                           [0.00001, 0.002  ],
                           [0.00001, 0.004  ],
                           [0.00001, 0.01  ],
                           [0.00001, 0.1  ],
                           [0.00001, 0],
                           [0.0001, 0],
                           [0.002, 0],
                           [0.003, 0],
                           [0.005, 0],
                           [0.007, 0],
                           [0.01  , 0],
                           [0.02  , 0],
                           [0.04  , 0],
                           [0.1  , 0],
                           [1.0    , 0]
                          ]:
            secreg = 0
            latentdim = 3
            epochs=800
            savedir,traintesterrors = train_vae_model(beta,gamma,lr,epochs,latentdim,testname,train_dataset,test_dataset,device=device)
            print(np.array(traintesterrors)[-10:,])
            all_models.append((beta,gamma,latentdim,lr,savedir))
            all_traintesterrors.append(traintesterrors)

overfitting/allposeslat3_lr5e-05_gamma0.001_es0.5_beta1e-05_20220716-224054
overfitting/allposeslat3_lr5e-05_gamma0.001_es0.5_beta1e-05_20220716-224054
[[2.31000000e+03 7.08513388e-02 7.20568599e-02]
 [2.32000000e+03 7.10493598e-02 7.17392296e-02]
 [2.33000000e+03 7.06402599e-02 7.20376444e-02]
 [2.34000000e+03 7.11439740e-02 7.33752288e-02]
 [2.35000000e+03 7.10738539e-02 7.16213391e-02]
 [2.36000000e+03 7.14889984e-02 7.25788609e-02]
 [2.37000000e+03 7.05370587e-02 7.33238818e-02]
 [2.38000000e+03 7.17805704e-02 7.43848674e-02]
 [2.39000000e+03 7.06790391e-02 7.24740187e-02]
 [2.40000000e+03 7.14951753e-02 7.61652288e-02]]
overfitting/allposeslat3_lr5e-05_gamma1e-05_es0.5_beta1e-05_20220716-224318
overfitting/allposeslat3_lr5e-05_gamma1e-05_es0.5_beta1e-05_20220716-224318
[[2.31000000e+03 1.45523812e-02 7.53435395e-02]
 [2.32000000e+03 1.46292833e-02 7.56128224e-02]
 [2.33000000e+03 1.46867558e-02 7.55769836e-02]
 [2.34000000e+03 1.44226998e-02 7.55774477e-02]
 [2.35000000e+03 1.4876

In [ ]:
np.savez(f'justquat_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}', all_models=all_models,all_traintesterrors=all_traintesterrors)

In [ ]:
dat = np.load("justquat_20220716-174754.npz")
all_models = dat["all_models"]
all_traintesterrors = dat["all_traintesterrors"]

In [ ]:
#savedirs = [
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta1e-05_20220714-180311",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta1e-05_20220714-180335",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta1e-05_20220714-180359",
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta0.001_20220714-180422",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta0.001_20220714-180447",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta0.001_20220714-180510",
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta1.0_20220714-180535",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta1.0_20220714-180558",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta1.0_20220714-180621"
#]

In [ ]:
all_traintesterrors

In [ ]:
all_models

In [ ]:
plt.figure()
for i,dat in enumerate(all_traintesterrors):
    (beta,gamma,latentdim,lr,savedir)=all_models[i]
    label="b%s_g%s" % (beta,gamma)
    dat = np.array(dat)
    okinds = (dat[:,1] < 0.3) * np.arange(len(dat)) > len(dat)*96/100
    train = dat[okinds,1]
    test = dat[okinds,2]
    if gamma == "0":
        plt.scatter(train,test,label=label,s=1,c="red")
    else:
        plt.scatter(train,test,label=label,s=1,c="k")
#plt.xlim(0,0.25)
#plt.ylim(0,0.25)
plt.plot([0,0.25],[0,0.25],linestyle="--")
for pcatrain, pcatest in [(0.10528128829504901, 0.10464429348233918),
                          (0.14522424739834378, 0.14321458052304378),
                         (0.22419371587397705, 0.2280505547284446)]:
    plt.scatter(pcatrain,pcatest,s=50,c="blue")
    plt.plot((0.01,0.2),(0.10464429348233918,0.10464429348233918))
#plt.legend()
plt.savefig("FittingJustQuaternionsAllPosesTrainSize100.png")

In [ ]:

plt.figure(figsize=(10,10))
for i,dat in enumerate(all_traintesterrors):
    (beta,gamma,latentdim,lr,savedir)=all_models[i]
    label="b%s_g%s" % (beta,gamma)
    dat = np.array(dat)
    okinds = (dat[:,1] < 0.3) * np.arange(len(dat)) > len(dat)*93/100
    train = dat[okinds,1]
    test = dat[okinds,2]
    if gamma != 0:
        plt.plot(np.ones_like(test) * gamma,test,label=label)
        plt.plot(np.ones_like(test) * gamma,train,label=label)
    else:
        pass#plt.plot(train,test,label=label)
plt.xscale("log")
plt.ylim(0,0.25)
for pcatrain, pcatest in [(0.08209298876189912, 0.08050582337947013),
                          (0.08209298876189912, 0.08050582337947013)]
    plt.plot([0.0001,0.1],[pcatrain,pcatrain],linestyle="--")
    plt.plot([0.0001,0.1],[pcatest,pcatest],linestyle="--")

plt.legend()